In [16]:
model_name = "russian-cursive-32x32rgb.model.keras"

tflite_model_name = "russian-cursive.tflite"

tflite_with_metadata_export_path = "./tflite_with_metadata"

class_names_file = "class_names-dataset.txt"

In [17]:
import keras
import tensorflow as tf

model = keras.models.load_model(model_name)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open(tflite_model_name, 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\argraur\AppData\Local\Temp\tmp2igbpi39\assets


INFO:tensorflow:Assets written to: C:\Users\argraur\AppData\Local\Temp\tmp2igbpi39\assets


In [18]:
%pip install tflite-support

Note: you may need to restart the kernel to use updated packages.


In [19]:
%mkdir .\tflite_with_metadata

import os
import tflite_support
from metadata_writer_for_image_classifier import MetadataPopulatorForImageClassifier
from metadata_writer_for_image_classifier import _MODEL_INFO

model_basename = os.path.basename(tflite_model_name)

export_model_path = os.path.join(tflite_with_metadata_export_path, model_basename)

tf.io.gfile.copy(tflite_model_name, export_model_path, overwrite=False)

populator = MetadataPopulatorForImageClassifier(export_model_path, _MODEL_INFO.get(model_basename), class_names_file)
populator.populate()

# Validate the output model file by reading the metadata and produce
# a json file with the metadata under the export path
displayer = tflite_support.metadata.MetadataDisplayer.with_model_file(export_model_path)
export_json_file = os.path.join(tflite_with_metadata_export_path,
                                os.path.splitext(model_basename)[0] + ".json")
json_file = displayer.get_metadata_json()

with open(export_json_file, "w") as f:
    f.write(json_file)